# Customer Churn Analysis – Data Cleaning & Preparation

This notebook cleans and prepares customer churn data for downstream analysis in SQL and visualization in Tableau.

Steps include:
- Data inspection
- Handling missing values
- Data type corrections
- Feature engineering
- Exporting cleaned data


In [2]:
import pandas as pd
import numpy as np

In [33]:
df1 = pd.read_excel('/Users/aaliyahgritly/Downloads/Bebo study items/(P4)/Telco_customer_churn.xlsx')
df = df1.copy()

In [34]:
pd.set_option('display.max_columns', None)
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [48]:
df["Paperless Billing"].value_counts()


Paperless Billing
Yes    4171
No     2872
Name: count, dtype: int64

In [35]:
df.shape

(7043, 33)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

Fix data types

In [37]:
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors = 'coerce') # Convert to numeric, setting errors to NaN

df['Total Charges'] = df['Total Charges'].fillna(0) # Fill NaN values with 0

In [38]:
df['Churn Reason'] = df['Churn Reason'].fillna("Active")  # Fill NaN values with "Active"

In [39]:
df['CustomerID'] = df['CustomerID'].astype(str)  # Convert CustomerID to string type

In [49]:
df['Churn Flag'] = df['Churn Value']

In [61]:
# Churn flag 1 = customer churned(left), 0 = customer is active
df["Churn Flag"].value_counts(normalize=True) * 100


Churn Flag
0    73.463013
1    26.536987
Name: proportion, dtype: float64

In [51]:
df.drop(columns = ['Churn Value'], inplace = True)

In [40]:
df['Senior Citizen'].value_counts()

Senior Citizen
No     5901
Yes    1142
Name: count, dtype: int64

Create tenure groups

In [ ]:
# tenure groups are whatever months the customer has been with the company
def tenure_groups(months):
    if months <= 12:
        return '0-12 Months'
    elif months <= 24:
        return '13-24 Months'
    elif months <= 48:
        return '25-48 Months'
    else:
        return '49+ Months'
    
df['Tenure Groups'] = df['Tenure Months'].apply(tenure_groups)

Group customers based on monthly charge

In [53]:
df['High Value Customer'] = np.where(df['Monthly Charges'] >= 70, 'Yes', 'No') # Identify high value customers based on Monthly Charges, threshold set at 70

Define number of services. Then, count how many services each customer is subscribed to.

In [54]:
# Define service-related columns
service_columns = ['Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 
                   'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies']

In [55]:
# sum(row == 'Yes') counts the number of 'Yes' in each row for the specified service columns
# axis = 1 indicates that the function is applied across columns for each row
df['Number of Services'] = df[service_columns].apply(lambda row: sum(row == 'Yes'), axis = 1)

In [73]:
df['churn_flag'].value_counts(normalize=True) * 100

churn_flag
0    73.463013
1    26.536987
Name: proportion, dtype: float64

In [57]:
df[['CustomerID', 'Number of Services', 'Monthly Charges']].head()

,CustomerID,Number of Services,Monthly Charges
0,3668-QPYBK,3,53.85
1,9237-HQITU,1,70.70
2,9305-CDSKC,5,99.65
3,7892-POOKP,6,104.80
4,0280-XJGEX,6,103.70


In [68]:
df['high_value_customer'].value_counts()

high_value_customer
Yes    3591
No     3452
Name: count, dtype: int64

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           7043 non-null   object 
 1   Count                7043 non-null   int64  
 2   Country              7043 non-null   object 
 3   State                7043 non-null   object 
 4   City                 7043 non-null   object 
 5   Zip Code             7043 non-null   int64  
 6   Lat Long             7043 non-null   object 
 7   Latitude             7043 non-null   float64
 8   Longitude            7043 non-null   float64
 9   Gender               7043 non-null   object 
 10  Senior Citizen       7043 non-null   object 
 11  Partner              7043 non-null   object 
 12  Dependents           7043 non-null   object 
 13  Tenure Months        7043 non-null   int64  
 14  Phone Service        7043 non-null   object 
 15  Multiple Lines       7043 non-null   o

Standardize column names

In [64]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [65]:
df.columns

Index(['customerid', 'count', 'country', 'state', 'city', 'zip_code',
       'lat_long', 'latitude', 'longitude', 'gender', 'senior_citizen',
       'partner', 'dependents', 'tenure_months', 'phone_service',
       'multiple_lines', 'internet_service', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'contract', 'paperless_billing', 'payment_method',
       'monthly_charges', 'total_charges', 'churn_label', 'churn_score',
       'cltv', 'churn_reason', 'churn_flag', 'tenure_groups',
       'high_value_customer', 'number_of_services'],
      dtype='object')

- export to csv

In [66]:
df.to_csv('/Users/aaliyahgritly/Downloads/Bebo study items/(P4)/churn_data_cleaned.csv', index = False)